## BIA Tischi demo
BIA Tischi is a Jupyter-magic based assistant for interacting with data via code

In [1]:
import numpy as np
from bia_tischi import bob
from bia_tischi._machinery import _context
_context.verbose = True
bob.__version__
bob.initialize(globals())
a = 5
%bob print out the variable a

Agent and context have been initialized already.
Variables: 20
Prompt ---- 
    Write concise professional bioimage analysis high-quality python code.
    The code should be as short as possible.
    If there are several ways to solve the task, chose the option with the least amount of code.
    Use preferably the python libraries scipy,sklearn,pandas,skimage,matplotlib,numpy,seaborn.
    Show results and save them in variables.
    The following variables are available: 
    Do not set the values of the variables that are available.
    The following functions are available: register_line_cell_magic,display,Markdown,generate_response,output_text,output_code,xbob,bob,update_context_and_fetch_user_input,CustomAgent,init_assistant
    A live python environment is available and the code you produce will be executed afterwards.

    The code block must be start with the line
    ```python
    and it must end with the line 
    ```

    Before writing the code, provide a concise step-by-ste

Step-by-Step Plan:

1. Start a new line of python code.
2. Use the print function to display the value of variable `a`.
3. Close the line of python code.

Here is the corresponding python code:

```python
print(a)
```

TypeError: Markdown expects text, not <IPython.core.display.Markdown object>

We initially give bob access to all variables in the current scope.

In [ ]:
bob.initialize(globals())

Turn on debug mode to see what happens

In [ ]:
# from bia_tischi._machinery import _context
# _context.verbose = True

It works using the `%tischi` and `%%tischi` magic commands. Just enter tasks and ask questions. Hint: if you want to ask a question, you need to put a space before the `?`.

In [ ]:
a = 5

In [ ]:
%bob print out the variable a

In [ ]:
%tischi load "blobs.tif", store it in the variable input_image and visualize it

In [ ]:
%%tischi 
* Use Otsu's method to segment the image stored in the variable input_image, 
* apply connected component labeling to label individual objects, 
* store the result in the variable label_image, and 
* visualize the result

In [ ]:
%tischi print out the number of objects in the label_image

In [ ]:
%tischi measure mean_intensity and major axis length of the objects in the input_image using the label_image as labels. Store the result in a dataframe called df

In [ ]:
%tischi show the dataframe df

In [ ]:
%tischi plot the mean_intensity against major_axis_length in the dataframe df 

In [ ]:
%tischi show a histogram of mean_intensity

In [ ]:
%%tischi 
please segment all nuclei in the blobs.tif fluorescence microscopy grayscale image 
and show the resulting label mask

In [ ]:
%%tischi 
please segment all nuclei in the blobs.tif grayscale image 
and show the resulting label mask